In [29]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [30]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
➡️⬆️🔄⬆️⬆️
➡️⏪⏬⬇️⬅️
➡️⬅️⏬🔄➡️
⬆️⏫️✅⏪⬆️
🔄🔄⬇️🔄⬆️


In [31]:
from IPython.display import clear_output

In [32]:
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.show()
gridworld.showPolicy(policy)
print("random policy")


trajectorySteps = 20000
epsilon = 0.1 #0.2
qtable = np.zeros((rows*columns,5))    #生成Qtable，也就是action-value-table

num_episodes = 200
for episode in range(num_episodes):
    # 
    if(epsilon > 0.001) :
        epsilon -= 0.001
    else:
        epsilon = 0.001
        
    p1 = 1-epsilon * (4/5)
    p0 = epsilon/5
    # trajectorySteps = int(20+epsilon*1000)
    print("trajectorySteps",trajectorySteps)
    print(f"epision:{epsilon}, p1:{p1}, p0:{p0}")
    
    d = {1:p1, 0:p0}
    policy_epsilon = np.vectorize(d.get)(policy)
    
    i = random.randint(0,24)  #初始状态
    j = random.randint(0,4)

    cnt = [0 for i in range(25)]
    qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)] 
    qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]
    Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy_epsilon, steps=trajectorySteps)
    clear_output(wait=True)
    # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
    score = 0
    for k in range(trajectorySteps,-1,-1):
        tmpstate, tmpaction, tmpscore, _, __  = Trajectory[k]
        cnt[tmpstate] += 1
        score = score*gamma + tmpscore  #细节从后往前优化算法
        
        qtable_rewards[tmpstate][tmpaction] += score
        qtable_nums[tmpstate][tmpaction] += 1
        qtable[tmpstate][tmpaction] = qtable_rewards[tmpstate][tmpaction] / qtable_nums[tmpstate][tmpaction]

    values = []
    for i in range(25):
        v = 0
        for j in range(5):
            v += policy_epsilon[i][j] * qtable[i][j]
        values.append(v)
    print(np.array(values).reshape(5,5))
    
    # print(qvalue.reshape(5,5))
    gridworld.showPolicy(policy)
    print(np.array(values).mean())
    
    
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    policy_epsilon = np.vectorize(d.get)(policy)
        
    # print(np.array(cnt).reshape(5,5))


    


    

[[-4.00551809e-04 -4.00035284e-03 -1.92287192e-01 -8.99659139e+00
  -9.98879557e+00]
 [-2.20293206e-03 -4.29060559e-03 -1.00645800e+00 -8.09714020e+00
  -9.97511497e+00]
 [-3.18021642e-04 -2.42876949e-03  9.99068284e+00 -4.00144532e-04
   6.47784629e+00]
 [-4.01056575e-04  9.98777194e+00  9.90913692e+00  9.99468079e+00
   7.19744115e+00]
 [-2.55517917e-03 -7.78886104e-03  9.96022892e+00  8.88578440e+00
   7.99718417e+00]]
➡️⬅️🔄➡️➡️
🔄⏬⏬⬆️➡️
➡️⬅️⏬⬆️⬇️
⬇️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
1.6847833442618705
